In [1]:
%%capture
!pip install langchain-community
!pip install pypdf
!pip install langchain-huggingface
!pip install chromadb
!pip install -U bitsandbytes

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `hff` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `hff`


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from transformers import pipeline
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig,AutoConfig
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

Latex Text Splitter

Embeddings

In [4]:
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [66]:
loader3=PyPDFLoader('Lora.pdf')
data3=loader3.load()
textSplitter=RecursiveCharacterTextSplitter(chunk_size=1800,chunk_overlap=450)
splits3=textSplitter.split_documents(data3)

In [67]:
doc_search=Chroma.from_documents(splits3,embeddings)

TASK 5 , RETRIEVER

In [68]:
retriever=doc_search.as_retriever(search_kwargs={"k":2})

In [8]:
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

In [9]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [10]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct",quantization_config=bnb_config)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [69]:
pipe=pipeline("text-generation",model=model,tokenizer=tokenizer,max_length=1000)

Device set to use cuda:0


In [50]:
from langchain.prompts import BasePromptTemplate
from langchain_core.prompt_values import PromptValue
from langchain_core.prompt_values import StringPromptValue
from typing import Dict, List, Any

In [51]:
class Llama3ChatPromptTemplate(BasePromptTemplate):
    tokenizer: Any  # Tokenizer for chat template

    def format(self, **kwargs: Any) -> str:
        """Format the prompt into a string"""
        context = kwargs.get("context", "")
        question = kwargs.get("question", "")

        messages = [
            {"role": "system", "content": "Use ONLY the context to answer. If unsure, say 'I don't know'."},
            {"role": "user", "content": f"CONTEXT:\n{context}\n\nQUESTION:\n{question}"}
        ]

        return self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

    def format_prompt(self, **kwargs: Any) -> PromptValue:
        """Format the prompt into a PromptValue"""
        return StringPromptValue(text=self.format(**kwargs))

    def _prompt_type(self) -> str:
        return "llama3_chat"

In [70]:
llama3_prompt = Llama3ChatPromptTemplate(
    input_variables=["context", "question"],
    tokenizer=tokenizer
)

In [71]:
from langchain.chains import RetrievalQA

In [72]:
qa=RetrievalQA.from_chain_type(
    llm=HuggingFacePipeline(pipeline=pipe),
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": llama3_prompt},
)

In [73]:
RetrievalQA.from_chain_type?

In [78]:
qa.invoke("Who is the author ?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'query': 'Who is the author ?',
 'result': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 20 Feb 2025\n\nUse ONLY the context to answer. If unsure, say 'I don't know'.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nCONTEXT:\nSQL query; in summarization, xi represents the article and yi would be its\nsummary.\nIn traditional fine-tuning, the model is initialized using the pre-trained weights\nΦ0, which are then updated to Φ 0 + ∆Φ by optimizing the model’s parameters\n\nof training. We then scale ∆ W xby α\nr , where α is a constant dependent on\nr. When using Adam for optimization, adjusting α has an effect similar to\ntuning the learning rate. Therefore, we use the same α for our first experiments\n\nQUESTION:\nWho is the author ?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI don't know."}